In [32]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence, List, Dict, Any
import operator
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage

In [33]:
llm = ChatOllama(model="llama3.2")

In [34]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    task: str
    plan: str
    critique: str


In [35]:
def researcher_node(state:AgentState):
    prompt = ("Role: Researcher. Propose 3 concise facts or sources to inform the solution. "
        "Output as bullet points. Do not provide a final answer.")    
    
    response = llm.invoke([SystemMessage(content=prompt), HumanMessage(content=state['task'])])
    return {"messages": [response]}

def planner_node(state:AgentState):
    is_revison = len(state.get("critique", "")) > 0
    role_instruction = "Role: Planner. Propose a ordered plan (3-5 steps) to solve the task."
    if is_revison:
        role_instruction += f"Revise the plan based on the critique provided. {state['critique']}"
    
    response = llm.invoke(
        [SystemMessage(content=role_instruction)] + state['messages'])
    return {"messages": [response], "plan": response.content}

def critic_node(state:AgentState):
    prompt = ("Role: Critic. Review the plan for gaps, conflicts or missing data. "
        "Return 2-3 actionable improvements.")
    
    response = llm.invoke(
        [SystemMessage(content=prompt)] + state['messages'])
    return {"messages": [response], "critique": response.content}

In [36]:
workflow = StateGraph(AgentState)

workflow.add_node("researcher", researcher_node)
workflow.add_node("planner", planner_node)
workflow.add_node("critic", critic_node)
workflow.add_node("reviser", planner_node)

In [37]:
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "planner")
workflow.add_edge("planner", "critic")
workflow.add_edge("critic", "reviser")
workflow.add_edge("reviser", END)

app = workflow.compile()

In [40]:
TASK = "Design a 3-email outreach sequence for a B2B AI tool targeting healthcare analytics leaders. Include subject lines and a clear CTA each."

inputs = {"task": TASK, "messages": [HumanMessage(content=TASK)], "plan": "", "critique": ""}

print("--- STARTING MULTI-AGENT RUN ---")
for output in app.stream(inputs):
    for node_name, state_update in output.items():
        print(f"\n[Node: {node_name.upper()}]")
        if "messages" in state_update:
            print(state_update["messages"][-1].content)

--- STARTING MULTI-AGENT RUN ---

[Node: RESEARCHER]
Here are three possible email sequences:

**Email 1: Introduction and Awareness (Subject Line: "Unlock the Power of AI in Healthcare Analytics")**

* Email Body:
	+ Introduce the AI tool and its benefits for healthcare analytics
	+ Highlight key statistics on the importance of data-driven decision making in healthcare
	+ Provide a brief overview of the upcoming email sequence
* CTA: "Get Instant Access to Our Free Whitepaper" (whitepaper will be sent via attachment or embedded in the email)
* Goal: Raise awareness and generate interest among the recipient

**Email 2: Problem-Agitate-Solve (Subject Line: "Are You Struggling with Data Quality in Your Analytics Team?")**

* Email Body:
	+ Identify a common pain point faced by healthcare analytics leaders (e.g. data quality issues)
	+ Agitate the problem by highlighting its consequences (e.g. delayed insights, incorrect decisions)
	+ Introduce the AI tool as a solution to address this is

In [42]:
final_state = app.invoke(inputs)

print("\n--- FINAL STATE ---")
print("Task:", final_state["task"])
print("Plan:", final_state["plan"])
print("Critique:", final_state["critique"])
print("Messages count:", len(final_state["messages"]))



--- FINAL STATE ---
Task: Design a 3-email outreach sequence for a B2B AI tool targeting healthcare analytics leaders. Include subject lines and a clear CTA each.
Plan: Each email is designed to be concise, clear, and relevant to the recipient's interests and pain points. The CTAs are also well-structured to encourage a response or action from the recipient.

Feel free to customize these examples to fit your specific brand voice and messaging!
Critique: 
Messages count: 5


In [55]:
final_state["messages"]

[HumanMessage(content='Design a 3-email outreach sequence for a B2B AI tool targeting healthcare analytics leaders. Include subject lines and a clear CTA each.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Here are three possible email outreach sequences:\n\n**Email 1: Introduction and Awareness**\n\nSubject Line: "Unlock the Power of Data-Driven Decision Making in Healthcare"\n\n* Brief introduction to the company and its B2B AI tool\n* Highlighting the importance of data analytics in healthcare decision making\n* Request for permission to connect with the recipient\'s organization\n\nExample email:\n\nDear [Recipient\'s Name],\n\nI came across your profile as a leader in the healthcare analytics space, and I\'d like to introduce you to our innovative B2B AI tool designed to help organizations like yours make data-driven decisions.\n\nOur solution leverages machine learning algorithms to analyze vast amounts of healthcare data, identifying trends, patterns, and in